# I. Imports

In [1]:
# External libraries

import numpy as np
import matplotlib.pyplot as plt

# Internal imports

from data import X, y, d
from aggregationFunctions import uniform_agg, gaussian_agg
from functions import *
from samplers import cmc
from variationalObjective import variational_obj
from gradientDescent import gradient_descent_W

# II. Sample from data using CMC

In [2]:
ncores = 10
W = np.array([np.eye(d) for k in range(ncores)])
betas = cmc(
    X = X,
    y = y,
    ncores=ncores,
    nsim=100,
    burnin=10
)

# III. Gradient descent

In [3]:
lr = 1
niter = 10
tol = 1e-6
sigma = 1

K,nsamples,d=betas.shape
S = np.array([np.mean([np.outer(betas[k,i],betas[k,i]) for i in range(nsamples)],axis=0) for k in range(K)])
mu = np.mean(betas,axis=1)
W = np.array([np.eye(d)/K for _ in range(K)]) #usual initialisation

W = gradient_descent_W(
    W_0 = W,
    S = S,
    mu = mu,
    betas = betas, 
    X = X,
    y = y,
    learning_rate = lr,
    niter = niter,
    tolerance = tol
)

iter: 0
iter: 1
iter: 2
iter: 3
iter: 4
iter: 5
iter: 6
iter: 7
iter: 8
iter: 9


In [6]:
betas_vcmc=np.sum([(W/np.sum(W,axis=0))[k] @np.mean(betas[k],axis=0) for k in range(K)],axis=0)
betas_vcmc/betas_vcmc[0]

array([ 1.        , -0.15523924, 79.12822357,  1.14072088,  0.96038183])

# Results

In [7]:
map_estimator=betas.mean(axis=(0,1))
map_estimator/=map_estimator[0]
map_estimator

array([ 1.        , -0.02943774,  0.01469499, -0.04778532, -0.94008352])

In [8]:
ncores_list=[5,25,50]
for ncores in ncores_list:
    sub_thetas=cmc(X,y,ncores=ncores,nsim=2000,burnin=500)
    theta_uni=uniform_agg(sub_thetas)
    theta_gauss=gaussian_agg(sub_thetas)
    theta_uni/=theta_uni[0]
    print(theta_uni)
    theta_gauss/=theta_gauss[0]
    print(theta_gauss)
    err_uni=np.linalg.norm(map_estimator-theta_uni,ord=1)/np.linalg.norm(map_estimator,ord=1)
    err_gauss=np.linalg.norm(map_estimator-theta_gauss,ord=1)/np.linalg.norm(map_estimator,ord=1)
    print("uni:",err_uni)
    print("gauss:",err_gauss)

[ 1.         -0.02297425  0.08488815 -0.19135351 -0.93081574]
[ 1.         -0.03536833  0.07736008 -0.19519449 -0.93858604]
uni: 0.11293918707873661
gauss: 0.10703845994848504
[ 1.          0.09800225  0.15415779  0.01626565 -0.75024872]
[ 1.          0.07337862  0.11776385 -0.00363788 -0.79347904]
uni: 0.25629338552027525
gauss: 0.19519529105134348
[ 1.          0.14310589  0.18850559  0.10636571 -0.54205388]
[ 1.          0.11047126  0.16189059  0.07760906 -0.62624053]
uni: 0.442192028108372
gauss: 0.35745147658948717


In [10]:
# y_hat_cmc=np.dot(X,map_cmc)>0
# np.count_nonzero(y_hat_cmc==y)/n